In [16]:
def parse_plate_layout(path_to_csv: str):
    """
    Parses CSV of the plate layout to JSON using GPT.
    1. Read the plate layout from the data folder as CSV
    2. Convert the CSV to JSON using GPT
    3. Save JSON to data folder
    """

    import os
    import openai
    import json

    # read CSV to string
    with open(path_to_csv, "r") as f:
        csv = f.read()

    # prepare prompt
    prompt = 'Here is a CSV file of a microplate layout: \n\n<CSV>\n\n----\n\nThere are 96 entries that map to measurement series of the plate reader (A-H)(1-12).\n\nReturn a JSON array which has the following properties for each well:\n{\n  "condition": str, // describes the condition (as seen in header row and column in CSV\n  "strain": str,\n  "replicate": str // replicate number as str, e.g. "1", "2", ..; each condition has multiple replicates as seen in the CSV cells  \n  "well": str // example values are "E12", "A1" and so on\n}\n\nThe JSON array should be sorted alphabetically by the well property. \n\nYou only return JSON, no text or anything else but the structured data. Return one array for data for all wells with letter A\n\nJSON\n-------\n'

    prompt_with_csv = prompt.replace("<CSV>", csv)

    # use GPT to convert CSV to JSON
    openai.api_key = os.getenv("OPENAI_API_KEY")

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt_with_csv,
        temperature=1,
        max_tokens=3000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["]"],
    )

    result = response.choices[0].text + "]"

    # validate that the response is valid JSON

    json.loads(result)

    # save JSON to data folder
    with open("data/plate_layout.json", "w") as f:
        f.write(result)

In [17]:
parse_plate_layout("data/plate_scheme.csv")

In [1]:
def assign_strains_to_wells(plate_layout):
    for el in plate_layout:
        if el["well"][0] in ["A", "E"]:
            el["strain"] = "MiniKat #1"
        elif el["well"][0] in ["B", "F"]:
            el["strain"] = "MiniKat p165"
        elif el["well"][0] in ["C", "G"]:
            el["strain"] = "MiniKat p183"
        elif el["well"][0] in ["D", "H"]:
            el["strain"] = "MiniKat p184"

    return plate_layout


def assign_conditions_to_wells(plate_layout):
    for el in plate_layout:
        if el["well"][1:] in ["1", "2"]:
            el["condition"] = "Ace30"
        elif el["well"][1:] in ["3", "4"]:
            el["condition"] = "Suc15"
        elif el["well"][1:] in ["5", "6"]:
            el["condition"] = "Gly20"
        elif el["well"][1:] in ["7", "8"]:
            el["condition"] = "Glc10"
        elif el["well"][1:] in ["9", "10"]:
            el["condition"] = "Glc10 + Ace10"
        elif el["well"][1:] in ["11", "12"] and el["well"][0] in ["A", "B", "C", "D"]:
            el["condition"] = "Glc40 + Ace10 + PQQ"
        elif el["well"][1:] in ["11", "12"] and el["well"][0] in ["E", "F", "G", "H"]:
            el["condition"] = "Glc10 + Acn10"

    return plate_layout


def assign_replicates_to_wells(plate_layout):
    for el in plate_layout:
        if el["well"][1:] in ["1", "3", "5", "7", "9", "11"]:
            el["replicate"] = "1"
        elif el["well"][1:] in ["2", "4", "6", "8", "10", "12"]:
            el["replicate"] = "2"

    return plate_layout


def assign_medium_to_wells(plate_layout):
    for el in plate_layout:
        if el["well"][0] in ["A", "B", "C", "D"]:
            el["medium"] = "MX"
        elif el["well"][0] in ["E", "F", "G", "H"]:
            if el["well"][1:] in ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]:
                el["medium"] = "MX + 250uM IPTG"
            else:
                el["medium"] = "MX"
    return plate_layout

In [2]:
import json

# read plate layout json
with open("data/plate_layout.json", "r") as f:
    plate_layout = json.load(f)

plate_layout = assign_strains_to_wells(plate_layout)
plate_layout = assign_conditions_to_wells(plate_layout)
plate_layout = assign_replicates_to_wells(plate_layout)
plate_layout = assign_medium_to_wells(plate_layout)

In [8]:
# save plate layout json
with open("data/plate_layout.json", "w") as f:
    json.dump(plate_layout, f)